匯入套件

In [1]:
import tensorflow as tf
import   keras 
from   keras.datasets   import   cifar10 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications   import   VGG19 
from keras.layers import Input

引用VGG19模型到model的變數，使用已訓練好的權重


In [2]:
vgg19_model = VGG19(include_top = True, weights='imagenet')
vgg19_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

複製VGG19層數,凍結VGG19的神經層，因為VGG19神經層已被ImageNet資料集訓練過，保留特徵 增添新的密集層，作為 cifar10 的10類分類器

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout


model = Sequential()


for layer in vgg19_model.layers[:-1]:
  model.add(layer)
  

for layer in model.layers:
  layer.trainable = False
  

model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

載入cifar10資料庫

In [5]:
(x_train, y_train) , (x_val, y_val) = cifar10.load_data()

引入資料預處理的套件

In [6]:


import numpy as np
import cv2



one-hot 標籤

In [7]:


import matplotlib.pyplot as plt

NUM_CLASSES = 10


y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_val = tf.keras.utils.to_categorical(y_val, NUM_CLASSES)



In [8]:
#新增getBatch函數(生成數量,取得訓練集、驗證集)
def getBatch(batch_size, train_or_val='train'):
#新建放大224x224的list
  x_batch = []
  y_batch = []
  if train_or_val == 'train':
    #隨機新增訓練集的數量
    idx = np.random.randint(0, len(x_train), (batch_size))
    #放大訓練集裡的圖片
    for i in idx:
      img = cv2.resize(x_train[i], (224, 224), interpolation=cv2.INTER_CUBIC)
      x_batch.append(img)
      y_batch.append(y_train[i])
  elif train_or_val == 'val':
     #隨機新增驗證集的數量
    idx = np.random.randint(0, len(x_val), (batch_size))
    #放大驗證集裡的圖片
    for i in idx:
      img = cv2.resize(x_val[i], (224, 224), interpolation=cv2.INTER_CUBIC)
      x_batch.append(img)
      y_batch.append(y_val[i]) 
  else:
    print("error, please specify train or val")
    #放入list
  x_batch = np.array(x_batch)
  y_batch = np.array(y_batch)
  return x_batch, y_batch
    



In [9]:
#compile函數定義損失函數(loss)、優化函數(optimizer)及成效衡量指標(mertrics)。
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [10]:
#ECHO數
EPOCHS = 20
#取張量數量
BATCH_SIZE = 50
#取訓練集數量
VAL_SIZE = 500
#做訓練集的次數
STEPS = 50


for e in range(EPOCHS):
  train_loss = 0
  train_acc = 0
    
  for s in range(STEPS):
    x_batch, y_batch = getBatch(BATCH_SIZE, "train")
    out = model.train_on_batch(x_batch, y_batch)
    train_loss += out[0]
    train_acc += out[1]
  
  print(f"Epoch: {e}\nTraining Loss = {train_loss / STEPS}\tTraining Acc = {train_acc / STEPS}")
  
  x_v, y_v = getBatch(VAL_SIZE, "val")
  eval = model.evaluate(x_v, y_v)
  print(f"Validation loss: {eval[0]}\tValidation Acc: {eval[1]}\n")
  
  





Epoch: 0
Training Loss = 1.992010930776596	Training Acc = 0.4544000019133091
16/16 [==============================] - 11s 485ms/step - loss: 0.9955 - categorical_accuracy: 0.6760
Validation loss: 0.9955011606216431	Validation Acc: 0.6759999990463257

Epoch: 1
Training Loss = 1.3663411331176758	Training Acc = 0.621200007200241
16/16 [==============================] - 6s 354ms/step - loss: 0.8781 - categorical_accuracy: 0.7260
Validation loss: 0.8780951499938965	Validation Acc: 0.7260000109672546

Epoch: 2
Training Loss = 1.0757788074016572	Training Acc = 0.685600004196167
16/16 [==============================] - 6s 360ms/step - loss: 0.8264 - categorical_accuracy: 0.7180
Validation loss: 0.8263738751411438	Validation Acc: 0.7179999947547913

Epoch: 3
Training Loss = 1.188940931558609	Training Acc = 0.6676000064611435
16/16 [==============================] - 6s 362ms/step - loss: 0.6953 - categorical_accuracy: 0.7720
Validation loss: 0.6952852606773376	Validation Acc: 0.7720000147819519


KeyboardInterrupt: 

In [ ]:
#辨識標籤
CATEGORIES = "airplane automobile bird cat deer dog frog horse ship truck".split(" ")

import matplotlib.pyplot as plt
#取10個隨機驗證集
x_v, y_v = getBatch(10, "val")

for i in range(10):

  import numpy as np
  plt.imshow(x_v[i])
  plt.show()
  print("pred: " +  CATEGORIES[np.argmax(model.predict(x_v[i:i+1]))])
  print("acct: " + CATEGORIES[np.argmax(y_v[i])])


 

